In [5]:
import os
import re
import time
import random
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup


In [7]:
BASE_URL = "https://oncology-association.ru/standards"
OUTPUT_DIR = "standards_pdfs"

def ensure_dir(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

def is_target_pdf_link(a_tag) -> bool:
    href = a_tag.get("href")
    if not href:
        return False
    text = (a_tag.get_text(strip=True) or "")
    # Текст должен начинаться с слова "Приказ" (регистр не важен)
    if not re.match(r"^\s*Приказ", text, re.IGNORECASE):
        return False
    full_url = urljoin(BASE_URL, href)
    # Проверяем, что ссылка ведёт на PDF
    return full_url.lower().endswith(".pdf")

def download_pdf(url: str, dest_path: str, session: requests.Session) -> None:
    with session.get(url, stream=True, timeout=20) as resp:
        resp.raise_for_status()
        total = int(resp.headers.get("Content-Length", 0))
        with open(dest_path, "wb") as f:
            downloaded = 0
            for chunk in resp.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)

def main():
    ensure_dir(OUTPUT_DIR)

    session = requests.Session()
    resp = session.get(BASE_URL, timeout=15)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    pdf_links = []
    for a in soup.find_all("a", href=True):
        if is_target_pdf_link(a):
            href = a["href"]
            full_url = urljoin(BASE_URL, href)
            text = a.get_text(strip=True)
            pdf_links.append((full_url, text))

    if not pdf_links:
        print("Не найдено ссылок на PDF с текстом, начинающимся на 'Приказ'.")
        return

    print(f"Найдено PDFs по приказам: {len(pdf_links)}")
    random.shuffle(pdf_links)

    for url, title in pdf_links:
        # "Без ограничений" по длине имени файла
        # Убираем недопустимые символы, но не обрезаем по длине
        # Замена любых недопустимых символов на подчеркивание
        sanitized_title = re.sub(r'[<>:"/\\|?*]+', '_', title)
        dest_filename = f"{sanitized_title}.pdf" if sanitized_title else "document.pdf"
        dest = os.path.join(OUTPUT_DIR, dest_filename)

        if os.path.exists(dest):
            print(f"Файл уже существует, пропуск: {dest}")
            continue

        # Вежливая задержка
        delay = random.uniform(1, 4)
        time.sleep(delay)

        try:
            download_pdf(url, dest, session)
            print(f"Скачан: {dest}")
        except Exception as e:
            print(f"Ошибка скачивания {url}: {e}")

if __name__ == "__main__":
    main()



Найдено PDFs по приказам: 18
Скачан: standards_pdfs/Приказ об утверждении стандартов медицинской помощи взрослым при экстракраниальных герминогенно-клеточных опухолях.pdf
Скачан: standards_pdfs/Приказ об утверждении стандартов медицинской помощи взрослым при раке вульвы.pdf
Ошибка скачивания https://oncology-association.ru/wp-content/uploads/2021/03/ob-utverzhdenii-standartov-mediczinskoj-pomoshhi-vzroslym-pri-vtorichnom-zlokachestvennom-novoobrazovanii-golovnogo-mozga-i-mozgovyh-obolochek.pdf: [Errno 36] File name too long: 'standards_pdfs/Приказ об утверждении стандартов медицинской помощи взрослым при вторичном злокачественном новообразовании головного мозга и мозговых оболочек.pdf'
Скачан: standards_pdfs/Приказ об утверждении стандартов медицинской помощи взрослым при опухолях средостения (диагностика и лечение).pdf
Скачан: standards_pdfs/Приказ об утверждении стандартов медицинской помощи мужчинам при герминогенных опухолях.pdf
Скачан: standards_pdfs/Приказ об утверждении стандарт